# Introduction to Convolutional Neural Networks

**Goal** : This notebook explains how to implement a simple convolutional neural network model. 

## MLP with Keras

In [1]:
import numpy as np
import pandas as pd
import os
from PIL import Image

### Load Data

In [2]:
#Load the data

labels = pd.read_csv("../data/BeeSpotter/train_labels.csv")
submissionFormat = pd.read_csv("../data/BeeSpotter/SubmissionFormat.csv")

In [3]:
labels.head()

,id,genus
0,520,1.0
1,3800,1.0
2,3289,1.0
3,2695,1.0
4,4922,1.0


In [4]:
submissionFormat.head()

,id,genus
0,2783,0.5
1,2175,0.5
2,4517,0.5
3,2831,0.5
4,3556,0.5


In [5]:
print submissionFormat.shape, labels.shape

(992, 2) (3969, 2)


##### Util to get an image. This will also help us resize the image after we read it

In [6]:
def get_image(filename, folder, size):
    filepath = os.path.join("../data/BeeSpotter/images", folder, filename + ".jpg")
    image = Image.open(filepath)
    image.thumbnail((size, size), Image.ANTIALIAS)
    return (np.array(image, dtype=np.int32)[:,:,:3]).swapaxes(0,2)

##### Util to load all the images. This will call the `get_image` function defined above

In [7]:
def load_images(filenames, folder, size):
    features = np.empty([len(filenames), 3, size, size])
    for counter, file in enumerate(filenames):
#        print file, counter
        features[counter] = get_image(str(file), folder, size)
    
    return features

##### Load the train and test datasets

In [8]:
train_X = load_images(labels.id, "train", 32)
test_X = load_images(submissionFormat.id, "test", 32)

In [9]:
print train_X.shape, test_X.shape

(3969, 3, 32, 32) (992, 3, 32, 32)


In [10]:
from keras.utils.np_utils import to_categorical
y_binary = to_categorical(np.array(labels.genus, dtype=int))

Using Theano backend.


##### Load `keras` package for CNN

In [11]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.backend import set_image_dim_ordering


set_image_dim_ordering('th')

##### First Model: Just one convolution layer

In [12]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, border_mode='full', input_shape=(3, 32, 32))) 
model.add(Flatten())

model.add(Dense(2, input_shape=(32*34*34,)))
model.add(Activation('sigmoid'))
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 32, 34, 34)    896         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 36992)         0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 2)             73986       flatten_1[0][0]                  
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 2)             0           dense_1[0][0]                    
Total params: 74882
_______________________________________________________________________

In [13]:
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)
model.fit(train_X, y_binary, batch_size=32, nb_epoch=10)

Epoch 1/10
3969/3969 [==============================] - 1s - loss: nan     
Epoch 2/10
3969/3969 [==============================] - 1s - loss: nan     
Epoch 3/10
3969/3969 [==============================] - 1s - loss: nan     
Epoch 4/10
3969/3969 [==============================] - 1s - loss: nan     
Epoch 5/10
3969/3969 [==============================] - 1s - loss: nan     
Epoch 6/10
3969/3969 [==============================] - 1s - loss: nan     
Epoch 7/10
3969/3969 [==============================] - 1s - loss: nan     
Epoch 8/10
3969/3969 [==============================] - 2s - loss: nan     
Epoch 9/10
3969/3969 [==============================] - 2s - loss: nan     
Epoch 10/10
3969/3969 [==============================] - 2s - loss: nan     


##### Second Model: One convolution layer and one RELU activation

In [14]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, border_mode='full', input_shape=(3, 32, 32))) 

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))

model.add(Flatten())

model.add(Dense(2, input_shape=(32*32*32,)))
model.add(Activation('sigmoid'))
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_2 (Convolution2D)  (None, 32, 34, 34)    896         convolution2d_input_2[0][0]      
____________________________________________________________________________________________________
convolution2d_3 (Convolution2D)  (None, 32, 32, 32)    9248        convolution2d_2[0][0]            
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 32, 32, 32)    0           convolution2d_3[0][0]            
____________________________________________________________________________________________________
flatten_2 (Flatten)              (None, 32768)         0           activation_2[0][0]               
___________________________________________________________________________________________

In [15]:
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)
model.fit(train_X, y_binary, batch_size=32, nb_epoch=10)

Epoch 1/10
3969/3969 [==============================] - 9s - loss: 0.7109     
Epoch 2/10
3969/3969 [==============================] - 10s - loss: 0.6931    
Epoch 3/10
3969/3969 [==============================] - 10s - loss: 0.6931    
Epoch 4/10
3969/3969 [==============================] - 10s - loss: 0.6931    
Epoch 5/10
3969/3969 [==============================] - 10s - loss: 0.6931    
Epoch 6/10
3969/3969 [==============================] - 10s - loss: 0.6931    
Epoch 7/10
3969/3969 [==============================] - 10s - loss: 0.6931    
Epoch 8/10
3969/3969 [==============================] - 11s - loss: 0.6931    
Epoch 9/10
3969/3969 [==============================] - 11s - loss: 0.6931    
Epoch 10/10
3969/3969 [==============================] - 10s - loss: 0.6931    


##### Third Model: One convolution layer, one RELU activation and one MaxPooling layer

In [16]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, border_mode='full', input_shape=(3, 32, 32))) 

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(2, input_shape=(32*16*16,)))
model.add(Activation('sigmoid'))
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_4 (Convolution2D)  (None, 32, 34, 34)    896         convolution2d_input_3[0][0]      
____________________________________________________________________________________________________
convolution2d_5 (Convolution2D)  (None, 32, 32, 32)    9248        convolution2d_4[0][0]            
____________________________________________________________________________________________________
activation_4 (Activation)        (None, 32, 32, 32)    0           convolution2d_5[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 32, 16, 16)    0           activation_4[0][0]               
___________________________________________________________________________________________

In [17]:
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)
model.fit(train_X, y_binary, batch_size=32, nb_epoch=10)

Epoch 1/10
3969/3969 [==============================] - 10s - loss: 3.3545    
Epoch 2/10
3969/3969 [==============================] - 10s - loss: 3.3584    
Epoch 3/10
3969/3969 [==============================] - 10s - loss: 3.3584    
Epoch 4/10
3969/3969 [==============================] - 10s - loss: 3.3584    
Epoch 5/10
3969/3969 [==============================] - 11s - loss: 3.3584    
Epoch 6/10
3969/3969 [==============================] - 10s - loss: 3.3584    
Epoch 7/10
3969/3969 [==============================] - 11s - loss: 3.3584    
Epoch 8/10
3969/3969 [==============================] - 11s - loss: 3.3584    
Epoch 9/10
3969/3969 [==============================] - 11s - loss: 3.3584    
Epoch 10/10
3969/3969 [==============================] - 11s - loss: 3.3584    


##### Fourth Model: One convolution layer, one RELU activation, one MaxPooling layer with dropout

In [18]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, border_mode='full', input_shape=(3, 32, 32))) 

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(2, input_shape=(32*16*16,)))
model.add(Activation('sigmoid'))
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_6 (Convolution2D)  (None, 32, 34, 34)    896         convolution2d_input_4[0][0]      
____________________________________________________________________________________________________
convolution2d_7 (Convolution2D)  (None, 32, 32, 32)    9248        convolution2d_6[0][0]            
____________________________________________________________________________________________________
activation_6 (Activation)        (None, 32, 32, 32)    0           convolution2d_7[0][0]            
____________________________________________________________________________________________________
maxpooling2d_2 (MaxPooling2D)    (None, 32, 16, 16)    0           activation_6[0][0]               
___________________________________________________________________________________________

In [19]:
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)
model.fit(train_X, y_binary, batch_size=32, nb_epoch=10)

Epoch 1/10
3969/3969 [==============================] - 11s - loss: nan    
Epoch 2/10
3969/3969 [==============================] - 10s - loss: nan    
Epoch 3/10
3969/3969 [==============================] - 11s - loss: nan    
Epoch 4/10
3969/3969 [==============================] - 10s - loss: nan    
Epoch 5/10
3969/3969 [==============================] - 11s - loss: nan    
Epoch 6/10
3969/3969 [==============================] - 11s - loss: nan    
Epoch 7/10
3969/3969 [==============================] - 11s - loss: nan    
Epoch 8/10
3969/3969 [==============================] - 11s - loss: nan    
Epoch 9/10
3969/3969 [==============================] - 11s - loss: nan    
Epoch 10/10
3969/3969 [==============================] - 11s - loss: nan    


##### A deep CNN

In [20]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, border_mode='full', input_shape=(3, 32, 32))) 
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 3, 3)) 
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, input_shape=(64*8*8,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(2, input_shape=(256,)))
model.add(Activation('sigmoid'))
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_8 (Convolution2D)  (None, 32, 34, 34)    896         convolution2d_input_5[0][0]      
____________________________________________________________________________________________________
activation_8 (Activation)        (None, 32, 34, 34)    0           convolution2d_8[0][0]            
____________________________________________________________________________________________________
convolution2d_9 (Convolution2D)  (None, 32, 32, 32)    9248        activation_8[0][0]               
____________________________________________________________________________________________________
activation_9 (Activation)        (None, 32, 32, 32)    0           convolution2d_9[0][0]            
___________________________________________________________________________________________

In [21]:
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)
model.fit(train_X, y_binary, batch_size=32, nb_epoch=10)

Epoch 1/10
3969/3969 [==============================] - 18s - loss: nan    
Epoch 2/10
3969/3969 [==============================] - 18s - loss: nan    
Epoch 3/10
3969/3969 [==============================] - 18s - loss: nan    
Epoch 4/10
3969/3969 [==============================] - 18s - loss: nan    
Epoch 5/10
3969/3969 [==============================] - 18s - loss: nan    
Epoch 6/10
3969/3969 [==============================] - 18s - loss: nan    
Epoch 7/10
3969/3969 [==============================] - 18s - loss: nan    
Epoch 8/10
3969/3969 [==============================] - 18s - loss: nan    
Epoch 9/10
3969/3969 [==============================] - 18s - loss: nan    
Epoch 10/10
3969/3969 [==============================] - 18s - loss: nan    
